In [6]:
# Extracting call logs from facebook data
# location of message_*.html files
# for privacy reasons folder name replaces with "namelastname"

path = 'D:/namelastname_10156896244312114'

In [32]:
import glob
import re
import pandas

In [8]:
files = glob.glob(path+'/*.html')

In [9]:
files

['D:/namelastname_10156896244312114\\message_1.html',
 'D:/namelastname_10156896244312114\\message_10.html',
 'D:/namelastname_10156896244312114\\message_11.html',
 'D:/namelastname_10156896244312114\\message_12.html',
 'D:/namelastname_10156896244312114\\message_2.html',
 'D:/namelastname_10156896244312114\\message_3.html',
 'D:/namelastname_10156896244312114\\message_4.html',
 'D:/namelastname_10156896244312114\\message_5.html',
 'D:/namelastname_10156896244312114\\message_6.html',
 'D:/namelastname_10156896244312114\\message_7.html',
 'D:/namelastname_10156896244312114\\message_8.html',
 'D:/namelastname_10156896244312114\\message_9.html']

In [10]:
from bs4 import BeautifulSoup

In [11]:
out = []
for f in files:

    with open(f, encoding="utf8") as my_file:
        s = str(my_file.read())
    soup = BeautifulSoup(s, 'html.parser')
    calls = soup.find_all('span',class_='_idm') 
    for c in calls:

        date = c.find_next('div',class_='_a72d')
        call_init = c.find_previous('div',class_='_2ph_ _a6-p')
        tmp = call_init.get_text()

        out.append((tmp[0:tmp.rfind('.')+1], c.get_text(),date.get_text() ))

In [12]:
import pandas as pd

In [16]:
df = pd.DataFrame(out)

In [24]:
df.sample(10)

,0,1,2
929,The video chat ended.,Duration: 28 minutes,"Sep 09, 2020 10:49:34am"
965,Name called you.,Duration: 41 minutes,"Aug 25, 2020 2:14:13am"
630,The video chat ended.,Duration: 24 minutes,"Aug 10, 2021 3:58:16pm"
963,You called Name.,Duration: 8 minutes,"Aug 25, 2020 1:39:13pm"
232,You called Name.,Duration: 1 minute,"Jun 21, 2020 11:37:09am"
754,You called Name.,Duration: 42 minutes,"Feb 13, 2021 3:25:12am"
451,The video chat ended.,Duration: 89 minutes,"Nov 05, 2021 5:48:07am"
159,Name called you.,Duration: 25 seconds,"Jul 24, 2022 3:58:40pm"
541,The video chat ended.,Duration: 29 minutes,"Sep 18, 2021 2:27:26pm"
897,The video chat ended.,Duration: 11 minutes,"Sep 25, 2020 4:32:31pm"


In [25]:
df.columns = ['Call','Duration','Date']
df = df[['Date','Call','Duration']]
df['Duration'] = df['Duration'].transform(lambda x: x.replace('Duration: ',''))

In [27]:
df['_date'] = pd.to_datetime(df['Date'])

In [29]:
# Duration is listed in either seconds or minutes

df['m'] = df['Duration'].str.contains('minute')

In [45]:
df['Duration'].sample(10)

939     120 minutes
128       2 minutes
874       2 minutes
370       2 minutes
277      31 minutes
412        1 minute
119      25 seconds
767     109 minutes
1003     36 minutes
51       51 seconds
Name: Duration, dtype: object

In [33]:
# extract numerical values
df['value']=df['Duration'].transform(lambda x: re.sub('[^0-9]', '', x)).astype(int)
    

In [34]:
df.sample(10)

,Call,Duration,Date,_date,m,value
490,The video chat ended.,3 minutes,"Oct 13, 2021 11:23:45am",2021-10-13 11:23:45,True,3
460,The video chat ended.,7 minutes,"Oct 30, 2021 11:00:02am",2021-10-30 11:00:02,True,7
510,You called Name.,1 minute,"Sep 27, 2021 12:10:42pm",2021-09-27 12:10:42,True,1
127,You called Name.,8 seconds,"Aug 30, 2022 5:58:50pm",2022-08-30 17:58:50,False,8
99,The video chat ended.,56 seconds,"Oct 16, 2022 4:13:50pm",2022-10-16 16:13:50,False,56
775,The video chat ended.,121 minutes,"Jan 25, 2021 5:58:42pm",2021-01-25 17:58:42,True,121
245,The video chat ended.,39 minutes,"Jun 08, 2020 1:37:43am",2020-06-08 01:37:43,True,39
643,The video chat ended.,9 minutes,"Aug 03, 2021 7:13:33pm",2021-08-03 19:13:33,True,9
982,Name called you.,15 seconds,"Aug 20, 2020 8:39:28pm",2020-08-20 20:39:28,False,15
70,You called Name.,35 seconds,"Nov 26, 2022 9:13:18pm",2022-11-26 21:13:18,False,35


In [35]:
# calculate total in seconds
total = 0
for m, v in zip(df['m'], df['value']):
    if m:
        total = total+v
    else:
        total = total+v*60

In [36]:
total

442005

In [37]:
import datetime

In [46]:
print('Total:', datetime.timedelta(seconds=total))

Total: 5 days, 2:46:45


In [39]:
# filter calls with duration>1 minutes
mdf = df[df['m']]

In [40]:
# filter calls with duration >=10 minutes
mdf = mdf[mdf['value']>=10]

In [41]:
mdf = mdf.sort_values(by='_date')[['Date','Call','Duration']]

In [43]:
mdf.sample(10)

,Date,Call,Duration
836,"Dec 11, 2020 2:05:51pm",The video chat ended.,37 minutes
274,"May 08, 2020 1:40:12am",The video chat ended.,19 minutes
520,"Sep 21, 2021 8:24:44pm",The video chat ended.,14 minutes
772,"Jan 28, 2021 12:16:59am",The video chat ended.,66 minutes
852,"Nov 24, 2020 12:49:13pm",You called Name.,35 minutes
439,"Nov 16, 2021 2:10:15am",Name called you.,25 minutes
875,"Oct 28, 2020 11:16:02pm",You called Name.,41 minutes
313,"Apr 07, 2020 3:07:20pm",The video chat ended.,72 minutes
896,"Sep 25, 2020 5:02:34pm",The video chat ended.,26 minutes
600,"Aug 20, 2021 11:47:29pm",The video chat ended.,83 minutes


In [163]:
mdf.to_csv('messenger_call_logs.csv',index=False)